In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model

### Define the file paths

In [2]:
train_file_path = 'train'
test_file_path = 'test'

### Load/resize the image from directory

In [3]:
image_size =(180, 180)

In [4]:
data_train = tf.keras.utils.image_dataset_from_directory(
    train_file_path,
    shuffle= True,
    image_size= image_size,
    batch_size=32,
    validation_split=0.2,
    seed=123,
    subset='training'
)

Found 557 files belonging to 2 classes.
Using 446 files for training.


In [5]:
data_val = tf.keras.utils.image_dataset_from_directory(
    train_file_path,
    shuffle= True,
    image_size= image_size,
    batch_size=32,
    validation_split=0.2,
    seed=123,
    subset='validation'
)

Found 557 files belonging to 2 classes.
Using 111 files for validation.


In [6]:
data_test = tf.keras.utils.image_dataset_from_directory(
    test_file_path,
    shuffle= False,
    image_size= image_size,
    batch_size=32,
    validation_split=False
)

Found 140 files belonging to 2 classes.


### Display the image classes

In [7]:
data_cat = data_train.class_names
data_cat

['cats', 'dogs']

### Apply rescaling to normalize pixel values

In [8]:
data_train = data_train.map(lambda x, y: (x / 255.0, y))
data_val = data_val.map(lambda x, y: (x / 255.0, y))
data_test = data_test.map(lambda x, y: (x / 255.0, y))

### Display the image classes

In [9]:
# data_train_list = list(data_train)
# img = random.randint(0, len(data_train_list) - 1)
# sample = data_train_list[img]  

# if len(sample[0].shape) == 4:  
#     single_image = sample[0][0] 
# else:
#     single_image = sample[0]

# plt.imshow(single_image)
# plt.show()


### Set up the model using functional api

In [9]:

# Define the input layer with the shape of the input images
input_shape = (180, 180, 3)  
input_layer = Input(shape=input_shape)

# Add convolutional and pooling layers
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

# Flatten the output of the convolutional layers
x = Flatten()(x)

# Define the fully connected layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Add the output layer
output_layer = Dense(1, activation='sigmoid')(x)  

In [10]:
# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# print model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       73856 

### Fit the model for training

In [12]:
model.fit(data_train, validation_data=data_val, epochs=15)

Epoch 1/15
14/14 [==============================] - 15s 894ms/step - loss: 0.9647 - accuracy: 0.4955 - val_loss: 0.7010 - val_accuracy: 0.4865
Epoch 2/15
14/14 [==============================] - 13s 866ms/step - loss: 0.6962 - accuracy: 0.4821 - val_loss: 0.6924 - val_accuracy: 0.4865
Epoch 3/15
14/14 [==============================] - 13s 887ms/step - loss: 0.6918 - accuracy: 0.5493 - val_loss: 0.6927 - val_accuracy: 0.4865
Epoch 4/15
14/14 [==============================] - 14s 932ms/step - loss: 0.6850 - accuracy: 0.5717 - val_loss: 0.6855 - val_accuracy: 0.5856
Epoch 5/15
14/14 [==============================] - 14s 947ms/step - loss: 0.6749 - accuracy: 0.6031 - val_loss: 0.6836 - val_accuracy: 0.5495
Epoch 6/15
14/14 [==============================] - 15s 977ms/step - loss: 0.6408 - accuracy: 0.6704 - val_loss: 0.6839 - val_accuracy: 0.5225
Epoch 7/15
14/14 [==============================] - 16s 1s/step - loss: 0.5692 - accuracy: 0.7354 - val_loss: 0.7034 - val_accuracy: 0.5766
Ep

### Predict the model

In [16]:
from PIL import Image

# Create an iterator
data_iter = iter(data_test)

# Get a random batch by iterating through the dataset
idx2 = random.randint(0, len(data_test) - 1)
for i in range(idx2 + 1):
    batch = next(data_iter)

# Select a random image from the batch
batch_size = batch[0].shape[0]
img_in_batch = random.randint(0, batch_size - 1)
single_image = batch[0][img_in_batch]

# Convert to numpy 
if isinstance(single_image, tf.Tensor):
    single_image = single_image.numpy()



# preprocess the image
image = Image.fromarray((single_image * 255).astype(np.uint8))  
image = tf.keras.utils.img_to_array(image)
image = tf.image.resize(image, (180, 180))  

# Expand dimensions to create a batch of one image
image_batch = tf.expand_dims(image, 0)

# Predict using the model
predict = model.predict(image_batch)
predict


1/1 [==============================] - 0s 76ms/step


array([[1.6779324e-16]], dtype=float32)

In [15]:
score = tf.nn.sigmoid(predict)
print("The animal in image is {} with accuracy of {:0.2f}".format(data_cat[np.argmax(score)], np.max(score)*100))

The animal in image is cats with accuracy of 50.00


### Save the model

In [17]:
model.save('animal prediction')

INFO:tensorflow:Assets written to: animal prediction\assets


INFO:tensorflow:Assets written to: animal prediction\assets
